In [ ]:
pip install ipynb

In [ ]:
from ipynb.fs.full.metrics import *
from ipynb.fs.full.util import *

In [ ]:
import csv 

def find_clusters(arr, speakers):
    active_mention = False
    active_replacement = False
    mention = []
    replacement = []
    finished = False
    finished_mention = False
    clusters = []
    num_of_sentence = 0
    length_of_additions = 0
    last_cluster = []
    last_replacement = []
    last_start_star_ind = 0
    i = 0
    for word in arr:
        if word == "." or word == "?" or word == "!":
            num_of_sentence += 1
        if word == "]]" and active_mention:
            length_of_additions += 1
            active_mention = False
            finished_mention = True
        if word == "**" and active_replacement:
            length_of_additions += 1
            active_replacement = False
            finished = True
            finished_mention = False
        if active_mention:
            mention.append(word)
        if active_replacement:
            replacement.append(word)
            length_of_additions += 1
        if word == "[[" and not active_mention:
            length_of_additions += 1
            active_mention = True
        if word == "**" and not finished and finished_mention:
            last_start_star_ind = i
            length_of_additions += 1
            active_replacement = True
        if word == "**" and not finished and not finished_mention:
            length_of_additions += 1
            k = last_start_star_ind
            new_rep = [item for item in arr[k+1:i] if item != "**"]
            clusters = [item for item in clusters if item != last_cluster]
            if last_cluster != []:
                cluster = [item for item in last_cluster[0] if item != last_replacement]
                cluster.append(new_rep)
                cluster = (cluster, last_cluster[1])
            else:
                cluster = []
                cluster.append(new_rep)
                cluster = (cluster, check_speaker(num_of_sentence, speakers))
            clusters.append(cluster)
        if finished:
            cluster_exists = False
            for cluster in clusters:
                if cluster[0][0] == replacement and cluster[1] == check_speaker(num_of_sentence, speakers):
                    cluster[0].append(mention)
                    cluster_exists = True
                    last_cluster = cluster
            if (cluster_exists == False):
                speaker = check_speaker(num_of_sentence, speakers)
                cluster = ([replacement, mention], speaker)
                last_cluster = cluster
                clusters.append(cluster)
            mention = []
            last_replacement = replacement
            replacement = []
            finished = False
        i += 1
    final_clusters = []
    for cluster in clusters:
        final_clusters.append(cluster[0])
    return final_clusters, length_of_additions


def print_original_and_predicted_clusters(og_clusters, pred_clusters):
    i = 0
    for cluster in og_clusters:
        if cluster == []:
            continue
        print("****ORIGINAL CLUSTER NUMBER " + str(i) + "****")
        for mention in cluster:
            print(mention)
        i += 1
    i = 0
    for cluster in pred_clusters:
        print("****PREDICTED CLUSTER NUMBER " + str(i) + "****")
        for mention in cluster:
            print(mention)
        i += 1 


def find_all_clusters_of_predictions(file, jsonlines_file):         
    f = open(file, newline='')
    reader = csv.reader(f)
    next(reader, None)
    number_of_lines = 0
    all_pred_clusters = []
    all_gold_clusters = []
    for row in reader:
        pred_text = row[2]
        og_text = row[0]
        split_og = format_the_string(og_text, True)
        split_pred = remove_values_from_array(format_the_string(pred_text), "pad")
        our_line = find_line(split_og, jsonlines_file)
        if our_line == {}:
            continue
        number_of_lines += 1
        all_words = get_all_words_from_sentences(our_line["sentences"])
        clusters, length_of_combined_replacements = find_clusters(split_pred, our_line["speakers"])
        gold_clusters = prune_gold_clusters(our_line["clusters"], len(split_pred)-length_of_combined_replacements)
        gold_clusters = turn_gold_cluster_indices_to_words(all_words, gold_clusters)
        all_pred_clusters.append(clusters)
        all_gold_clusters.append(gold_clusters)
    return all_pred_clusters, all_gold_clusters


file = "dev_outputs.csv"
jsonlines_file = "dev.english.jsonlines"
to_print = True
def metrics_of_predictions(file, jsonlines_file, to_print):
    all_pred_clusters, all_gold_clusters = find_all_clusters_of_predictions(file, jsonlines_file)
    return calculate_and_print_metrics_of_all_clusters(all_pred_clusters, all_gold_clusters, to_print)        


metrics_of_predictions('dev_outputs.csv', 'dev.english.jsonlines', to_print = True)